In [0]:
#!conda install -c conda-forge jsonnet
pip install allennlp

     |████████████████████████████████| 5.7MB 3.0MB/s 
     |████████████████████████████████| 1.6MB 36.7MB/s 
     |████████████████████████████████| 51kB 26.8MB/s 
     |████████████████████████████████| 245kB 59.3MB/s 
     |████████████████████████████████| 51kB 24.9MB/s 
     |████████████████████████████████| 256kB 50.0MB/s 
     |████████████████████████████████| 245kB 38.7MB/s 
     |████████████████████████████████| 133kB 48.5MB/s 
     |████████████████████████████████| 51kB 25.0MB/s 
     |████████████████████████████████| 5.5MB 53.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/b7/8d/e7/a0e74217da5caeb3c1c7689639b6d28ddbf9985b840bc96a9a
  Stored in directory: /root/.cache/pip/wheels/1a/30/ab/ae4a57b1df44fa20a531edb9601b27603da8f5336225691f3f
  Stored in directory: /root/.cache/pip/wheels/51/30/d1/92a39ba40f21cb70e53f8af96eb98f002a781843c065406500
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
  Stored in dire

In [0]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.optim as optim
import torch.backends.cudnn as cudnn

from typing import *
from pathlib import Path

from allennlp.data.dataset_readers import LanguageModelingReader
from allennlp.data.tokenizers import CharacterTokenizer
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data import Vocabulary
from allennlp.data.iterators import BasicIterator
from allennlp.training import Trainer
from sklearn.model_selection import train_test_split
from allennlp.modules.seq2seq_encoders import PytorchSeq2SeqWrapper
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.models import Model
from allennlp.nn.util import get_text_field_mask

import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [0]:
class LanguageModel(Model):
    def __init__(self, encoder, vocab,
                 embedding_dim=50):
        super().__init__(vocab=vocab)
        # char embedding
        self.vocab_size = vocab.get_vocab_size()
        self.padding_idx = vocab.get_token_index("@@PADDING@@")
        token_embedding = Embedding(
            num_embeddings=vocab.get_vocab_size(),
            embedding_dim=embedding_dim,
            padding_index=self.padding_idx,
        )
        self.embedding = BasicTextFieldEmbedder({"tokens": token_embedding})
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.hidden_size, self.vocab_size)
        self.loss = nn.CrossEntropyLoss(ignore_index=self.padding_idx)
    
    def forward(self, input_tokens,
                output_tokens):
        embs = self.embedding(input_tokens)
        x, _ = self.encoder(embs)
        x = self.projection(x)
        if output_tokens is not None:
            loss = self.loss(x.view((-1, self.vocab_size)), output_tokens["tokens"].flatten())
        else:
            loss = None
        return {"loss": loss, "logits": x}

In [0]:
# initialization of model weights
def init_Ws(parameters):
    for p in parameters:
        if p.data.ndimension() > 1:
            nn.init.xavier_uniform_(p.data)
        else:
            nn.init.zeros_(p.data)

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size, self.hidden_size = input_size, hidden_size

        # define nn.Parameter for all the parameters in RNN
        self.W_ih = Parameter(torch.Tensor(input_size, hidden_size))
        self.W_hh = Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_h = Parameter(torch.Tensor(hidden_size))

        init_Ws(self.parameters())
    
    def forward(self, x, init_state=None):
        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_size, _ = x.size()
        hidden_seq = []
        if init_state is None:
            h_t = torch.zeros(self.hidden_size).to(x.device)
        else:
            h_t = init_state

        # BEGIN OF TODO ----------------------------------------------
        for t in range(seq_size):
            x_t = x[:,t,:]
            
            h_t = torch.tanh((x_t @ self.W_ih) + (h_t @ self.W_hh) + self.b_h)
            hidden_seq.append(h_t.unsqueeze(0))
            
        # END OF TODO ----------------------------------------------

        hidden_seq = torch.cat(hidden_seq, dim=0)
        # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        return hidden_seq, h_t

In [0]:
class naiveLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # BEGIN OF TODO ----------------------------------------------
        # define nn.Parameter for all the parameters in LSTM
        # input gate
         # define nn.Parameter for all the parameters in RNN
        self.W_ii = Parameter(torch.Tensor(input_size, hidden_size))
        self.W_hi = Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_i = Parameter(torch.Tensor(hidden_size))
        # forget gate
        self.W_if = Parameter(torch.Tensor(input_size, hidden_size))
        self.W_hf = Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_f = Parameter(torch.Tensor(hidden_size))
        # compressed
        self.W_ig = Parameter(torch.Tensor(input_size, hidden_size))
        self.W_hg = Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_g = Parameter(torch.Tensor(hidden_size))

        # output gate
        self.W_io = Parameter(torch.Tensor(input_size, hidden_size))
        self.W_ho = Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_o = Parameter(torch.Tensor(hidden_size))

        # END OF TODO ----------------------------------------------
        
        init_Ws(self.parameters())
        
    def forward(self, x, init_states=None):
        """Assumes x is of shape (batch, sequence, feature)"""
        bs, seq_size, _ = x.size()
        hidden_seq = []
        if init_states is None:
            h_t, c_t = torch.zeros(self.hidden_size).to(x.device), torch.zeros(self.hidden_size).to(x.device)
        else:
            h_t, c_t = init_states

        # BEGIN OF TODO ----------------------------------------------
        for t in range(seq_size):
            x_t = x[:,t,:]
            i_t = torch.sigmoid((x_t @ self.W_ii) + (h_t @ self.W_hi) + self.b_i)
            f_t = torch.sigmoid((x_t @ self.W_if) + (h_t @ self.W_hf) + self.b_f)
            g_t = torch.tanh((x_t @ self.W_ig) + (h_t @ self.W_hg) + self.b_g)
            o_t = torch.sigmoid((x_t @ self.W_io) + (h_t @ self.W_ho) + self.b_o)
            c_t = f_t * c_t + i_t * g_t
            h_t = o_t * torch.tanh(c_t)
            hidden_seq.append(h_t.unsqueeze(0))
        # END OF TODO ----------------------------------------------

        hidden_seq = torch.cat(hidden_seq, dim=0)
        # reshape from shape (sequence, batch, feature) to (batch, sequence, feature)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()
        return hidden_seq, (h_t, c_t)

In [0]:
def main():

    DATA_ROOT = Path("./")

    char_tokenizer = CharacterTokenizer(lowercase_characters=True)

    reader = LanguageModelingReader(
        tokens_per_instance=500,
        tokenizer=char_tokenizer,
        token_indexers = {"tokens": SingleIdTokenIndexer()},
    )

    train_ds = reader.read(DATA_ROOT / "brown.txt")
    train_ds, val_ds = train_test_split(train_ds, random_state=0, test_size=0.1)

    vocab = Vocabulary.from_instances(train_ds)

    iterator = BasicIterator(batch_size=32)
    iterator.index_with(vocab)


    # Question 1 -----------------------------------------------------------------
    def train(model, epochs):
        trainer = Trainer(
            model=model.cuda() if torch.cuda.is_available() else model,
            optimizer=optim.Adam(model.parameters()),
            iterator=iterator, train_dataset=train_ds, 
            validation_dataset=val_ds, num_epochs=epochs,
            cuda_device=0 if torch.cuda.is_available() else -1
        )
        return trainer.train()

    lm_pytorchRNN = LanguageModel(nn.RNN(50, 125, batch_first=True), vocab)
    train(lm_pytorchRNN, epochs=30)

    lm_pytorchLSTM = LanguageModel(nn.LSTM(50, 125, batch_first=True), vocab)
    train(lm_pytorchLSTM, epochs=30)
    
    # Question 1 -----------------------------------------------------------------

    
    # Question 2 & 3 -----------------------------------------------------------------
    def train_q2(model, epochs):
        trainer = Trainer(
            model,
            optimizer=optim.Adam(model.parameters()),
            iterator=iterator, train_dataset=train_ds, 
            validation_dataset=val_ds, num_epochs=epochs,
            cuda_device=-1
        )
        return trainer.train()

    lm_RNN = LanguageModel(RNN(50, 125), vocab)
    train_q2(lm_RNN, epochs=1)

    lm_naiveLSTM = LanguageModel(naiveLSTM(50, 125), vocab)
    train_q2(lm_naiveLSTM, epochs=1)
    
    # Question 2 & 3 -----------------------------------------------------------------
    # Question 4 -----------------------------------------------------------------
    test_batch = next(iterator(train_ds))
    test_embeddings = lm_naiveLSTM.embedding(test_batch["input_tokens"])
    #print(test_embeddings.size())
    rnn = RNN(50, 125)
    h_0 = torch.zeros(rnn.hidden_size, requires_grad=True).to(test_embeddings.device)
    h_t = h_0
    grads = []
    #print(test_embeddings.size())
    for t in range(100):
        x_t = test_embeddings[:,t,:]
        # BEGIN OF TODO ----------------------------------------------
        # one tep of RNN
        x_t = test_embeddings[:,t,:]
        h_t = torch.tanh((x_t @ rnn.W_ih) + (h_t @ rnn.W_hh) + rnn.b_h)   
        
        # END OF TODO ----------------------------------------------
        loss = h_t.abs().sum()
        loss.backward(retain_graph=True)
        grads.append(torch.norm(h_0.grad).item())
        h_0.grad.zero_()
        rnn.zero_grad()

    #print(grads)
    plt.plot(grads)
    plt.xlabel("time steps")
    plt.ylabel("loss")
    plt.show()
    
    # Question 4 -----------------------------------------------------------------

    # Question 5 & 6-----------------------------------------------------------------
    # LSTM gradient with forget gate (three settings for forget gate, three settings for input gate)

    lstm = naiveLSTM(50, 125)

    # generate 
    hidden_size = lstm.hidden_size
    h_0, c_0 = (torch.zeros(hidden_size, requires_grad=True), 
                torch.zeros(hidden_size, requires_grad=True))
    grads = []
    h_t, c_t = h_0, c_0
    
    # BEGIN OF TODO ----------------------------------------------
    # Question 5
    # 1) without forget gate:
    
    

    # 2) with forget gate, b_f = 1

    # 3) with forget gate, b_f = 0

    # Question 6
    # 1) without input gate:

    # 2) with input gate, b_f = 1

    # 3) with input gate, b_f = 0

    # END OF TODO ----------------------------------------------
    grads = []
    new_b_i= torch.zeros(lstm.b_i.size())
    for t in range(100):

        # BEGIN OF TODO --------Different experiments by setting biases on input and forget gates to different values.---------------
        x_t = test_embeddings[:,t,:]
        i_t = torch.sigmoid((x_t @ lstm.W_ii) + (h_t @ lstm.W_hi) + new_b_i)
        f_t = torch.sigmoid((x_t @ lstm.W_if) + (h_t @ lstm.W_hf) + lstm.b_f)
        g_t = torch.tanh((x_t @ lstm.W_ig) + (h_t @ lstm.W_hg) + lstm.b_g)
        o_t = torch.sigmoid((x_t @ lstm.W_io) + (h_t @ lstm.W_ho) + lstm.b_o)
        c_t = f_t * c_t + i_t * g_t
        h_t = o_t * torch.tanh(c_t)
        # END OF TODO ----------------------------------------------
        loss = h_t.abs().sum()
        loss.backward(retain_graph=True)
        grads.append(torch.norm(h_0.grad).item())
        h_0.grad.zero_()
        lstm.zero_grad()
    print(grads)
    plt.plot(grads)
    plt.xlabel("time steps")
    plt.ylabel("loss")
    plt.show()

    
    

In [0]:
if __name__ == '__main__':
    main()
    



0it [00:00, ?it/s]

  0%|          | 0/11994 [00:00<?, ?it/s]

 37%|███▋      | 4464/11994 [00:00<00:00, 43343.45it/s]

 49%|████▊     | 5843/11994 [00:00<00:00, 25349.68it/s]

 84%|████████▎ | 10034/11994 [00:00<00:00, 28400.16it/s]

100%|██████████| 11994/11994 [00:00<00:00, 33182.86it/s]
1it [00:14, 14.43s/it]
169it [00:14, 10.10s/it]
612it [00:14,  7.07s/it]
1034it [00:14,  4.95s/it]
1405it [00:14,  3.47s/it]
1833it [00:14,  2.43s/it]
2196it [00:15,  1.70s/it]
2624it [00:15,  1.19s/it]
2992it [00:15,  1.20it/s]
3423it [00:15,  1.72it/s]
3856it [00:15,  2.45it/s]
4255it [00:15,  3.50it/s]
4679it [00:15,  5.00it/s]
5080it [00:15,  7.14it/s]
5510it [00:15, 10.19it/s]
5913it [00:16, 14.54it/s]
6347it [00:16, 20.74it/s]
6754it [00:16, 29.56it/s]
7190it [00:16, 42.10it/s]
7610it [00:16, 59.89it/s]
8022it [00:16, 84.97it/s]
8455it [00:16, 120.38it/s]
8865it [00:16, 169.66it/s]
9289it [00:16, 238.28it/s]
9697it [00:16, 331.21it/s]
10132it [00:17, 458.21it/s]
10540it [00:17, 621.92it/s]
1